# **Description**

**Housing Values in Suburbs of Boston**


*  The medv variable is the target variable.



**Data description**


*  The Boston data frame has 506 rows and 14 columns.




In [26]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision
import torch.nn as nn
import pandas as pd
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [27]:
# downloading datasets
DATASET_URL = "https://raw.githubusercontent.com/selva86/datasets/master/BostonHousing.csv"
DATA_FILENAME = "BostonHousing.csv"
TARGET_COLUMN = 'medv'
input_feature = 13
output_feature = 1

download_url(DATASET_URL,'.')
dataframe = pd.read_csv(DATA_FILENAME)
dataframe.head()

Using downloaded and verified file: ./BostonHousing.csv


,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [28]:
# converting the pandas dataframe to numpy arrays
inputs = dataframe.drop(TARGET_COLUMN, axis=1).values
targets = dataframe[[TARGET_COLUMN]].values

print(inputs.shape)
print(type(inputs))
print(targets.shape)
print(type(targets))

# we can also use the dataframe.to_numpy() ---> to convert complete panda dataset into numpy
# numpy_dataframe = dataframe.to_numpy()
# print(type(numpy_dataframe))

(506, 13)
<class 'numpy.ndarray'>
(506, 1)
<class 'numpy.ndarray'>


In [29]:

# converting the dataset into the Tensor
dataset = TensorDataset(torch.tensor(inputs, dtype= torch.float32), torch.tensor(targets, dtype=torch.float32))
train_ds, val_ds = random_split(dataset, [406, 100])

batch_size = 65

train_loader = DataLoader(train_ds, batch_size, shuffle= True)
val_loader = DataLoader(val_ds, batch_size, shuffle= True)

In [30]:
class HousingPredicationModel(nn.Module):
  
  def __init__(self) -> None:
      super().__init__()
      self.linear = nn.Linear(input_feature, output_feature) # it will create the model parameter matrixes (weight and bias matrix in this case)

  def forward(self, x):
    return self.linear(x)               # y = w@x + b ---> will return the y values
  
  def training_step(self, batch):
    inputs, targets = batch
    predication = self(inputs)         # it will call the __call__magic function, which further will call the forward function--> which will call the self.linear(inputs)
    loss = F.mse_loss(predication, targets)
    return loss

  def validation_step(self, batch):
    inputs, targets = batch
    predication = self(inputs)
    loss = F.mse_loss(predication, targets)
    return {'val_loss': loss.detach()}
  
  def validation_epoch_end(self, result):
    batch_losses = [x['val_loss'] for x in result]
    total_epoch_loss = torch.stack(batch_losses).mean()   # combine all the losses
    return({'val_loss': total_epoch_loss.item()})
  
  def print_result(self, epoch, final_result):
    print("Epoch [{}], val_loss: {:.4f}".format(epoch, final_result['val_loss']))

In [31]:
model = HousingPredicationModel()

In [32]:

def evaluate(model, val_loader):
  result = [model.validation_step(batch) for batch in val_loader]
  final_loss_value = model.validation_epoch_end(result)
  return final_loss_value

def fit(no_of_epochs, lr, model, train_loader, val_loader, opt_func= torch.optim.SGD):
  optimizer = opt_func(model.parameters(), lr)
  history = []

  for epoch in range(no_of_epochs):

    for batch in train_loader:
      loss = model.training_step(batch)
      loss.backward()                    # calculate the grad i.e w.grad() and b.grad()
      optimizer.step()                   # update the model parameters
      optimizer.zero_grad()              # zeroing the w.grad() and b.grad() matrices

    # validation Phase
    result = evaluate(model, val_loader)
    model.print_result(epoch, result)
    history.append(result)
  
  return history

In [33]:
result_0 = evaluate(model, val_loader)
result_0

{'val_loss': 11936.388671875}